In [5]:
def load_dataset(file_name):
    import pandas as pd
    import json
    
    df = []
    with open('/mnt/ceph/storage/data-in-progress/data-research/web-search/ECIR-22/ecir22-clickbait-spoiling/' + file_name) as f:
        for i in f:
            i = json.loads(i)
            tweet = i['postText']
            article_title = i['targetTitle']
            article = ' '.join(i['targetParagraphs'])
            label = i['tags']
            
            assert len(tweet) == 1
            tweet = tweet[0]
            
            assert len(label) == 1
            label = label[0]
            
            if label not in ['phrase', 'phrases', 'passage', 'multi']:
                print(label)
                
            assert label in ['phrase', 'phrases', 'passage', 'multi']
            
            if label == 'multi':
                continue
            
            df += [{'text': tweet + ' - ' + article_title + article, 
                    'labels': (label == 'phrase' or label == 'phrases')}]

    return pd.DataFrame(df)   
            
    
test_dataset = load_dataset('test.jsonl')
train_dataset = load_dataset('train.jsonl')
validation_dataset = load_dataset('validation.jsonl')

In [2]:
configurations = []

for learn_rate in [4e-5, 1e-4]:
    for warumup_ratio in [0.06, 0.1]:
        for batch_size in [8]:
            for seq_length in [256, 384, 512]:
                configurations += [{
                    "overwrite_output_dir": True,
                    "num_train_epochs": 10,
                    "fp16": False,
                    "train_batch_size": batch_size,
                    "gradient_accumulation_steps": 4,
                    "evaluate_during_training": True,
                    "max_seq_length": seq_length,
                    "learning_rate": learn_rate,
                    "early_stopping_consider_epochs": True,
                    "early_stopping_delta": 0.01,
                    "early_stopping_metric": "acc",
                    "early_stopping_metric_minimize": False,
                    "early_stopping_patience": 3,
                    "evaluate_during_training_steps": 331,
                    "output_dir": "outputs/",
                    'warmup_ratio': warumup_ratio,
                    # 'warmup_steps': 0, # usually empfohlen, warmup_ratio ist prozentuales äquivalent 
                    #                    # <> überschreiben sich gegenseitig
                    'save_steps': 2000,
                    "manual_seed": 12345
                }]

In [7]:
def train_model(config, num):
    from simpletransformers.classification import ClassificationModel
    import sklearn
    
    config["output_dir"] = "outputs/bert_concat_" + str(num)
    
    model = ClassificationModel("bert", "bert-large-cased", args = config)
    # model.train_model(train_dataset)
    model.train_model(train_dataset, eval_df=validation_dataset, acc=sklearn.metrics.accuracy_score)

In [ ]:
for config, num in zip(configurations, range(len(configurations))):
    print('Running configuration number', num)
    
    train_model(config, num)

Running configuration number 0


Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at 

  0%|          | 0/2641 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/331 [00:00<?, ?it/s]

In [13]:
from simpletransformers.classification import ClassificationModel

model = ClassificationModel("bert", "bert-large-cased", args = args)
# model.train_model(train_dataset)
model.train_model(train_dataset, eval_df=validation_dataset, acc=sklearn.metrics.accuracy_score)

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at 

RuntimeError: CUDA out of memory. Tried to allocate 114.00 MiB (GPU 0; 39.59 GiB total capacity; 37.38 GiB already allocated; 25.69 MiB free; 37.54 GiB reserved in total by PyTorch)

In [3]:
from glob import glob
from simpletransformers.classification import ClassificationModel
import sklearn
import pandas as pd
from tqdm import tqdm

df = []

for config, num in zip(configurations, range(len(configurations))):
    directory = "outputs/bert_concat_" + str(num) + "/"
    
    for checkpoint in tqdm(glob(directory + "checkpoint-*")):
        # print(checkpoint)
        model = ClassificationModel("bert", checkpoint)
        
        valid_acc = model.eval_model(validation_dataset, acc=sklearn.metrics.accuracy_score)[0]['acc']
        test_acc = model.eval_model(test_dataset, acc=sklearn.metrics.accuracy_score)[0]['acc']
        
        df += [{"checkpoint": checkpoint, "valid_acc": valid_acc, "test_acc": test_acc,
               "config": config}]
        
df = pd.DataFrame(df)
df

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/657 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/83 [00:00<?, ?it/s]

  0%|          | 0/826 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/104 [00:00<?, ?it/s]

...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


  0%|          | 0/826 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/104 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
100%|██████████| 12/12 [10:09<00:00, 50.78s/it]


,checkpoint,valid_acc,test_acc,config
0,outputs/bert_concat_0/checkpoint-246-epoch-3,0.700152,0.733656,"{'overwrite_output_dir': True, 'num_train_epoc..."
1,outputs/bert_concat_0/checkpoint-656-epoch-8,0.741248,0.751816,"{'overwrite_output_dir': True, 'num_train_epoc..."
2,outputs/bert_concat_0/checkpoint-820-epoch-10,0.736682,0.738499,"{'overwrite_output_dir': True, 'num_train_epoc..."
3,outputs/bert_concat_0/checkpoint-410-epoch-5,0.735160,0.722760,"{'overwrite_output_dir': True, 'num_train_epoc..."
4,outputs/bert_concat_0/checkpoint-331,0.739726,0.753027,"{'overwrite_output_dir': True, 'num_train_epoc..."
...,...,...,...,...
139,outputs/bert_concat_11/checkpoint-328-epoch-4,0.509893,0.512107,"{'overwrite_output_dir': True, 'num_train_epoc..."
140,outputs/bert_concat_11/checkpoint-738-epoch-9,0.509893,0.512107,"{'overwrite_output_dir': True, 'num_train_epoc..."
141,outputs/bert_concat_11/checkpoint-492-epoch-6,0.490107,0.487893,"{'overwrite_output_dir': True, 'num_train_epoc..."
142,outputs/bert_concat_11/checkpoint-574-epoch-7,0.509893,0.512107,"{'overwrite_output_dir': True, 'num_train_epoc..."


In [4]:
df.to_json('outputs/bert_concat_eval_results.csv')

In [2]:
import pandas as pd

df = pd.read_json('outputs/bert_concat_eval_results.csv')

In [3]:
df[df['checkpoint'].str.contains('epoch')].sort_values('valid_acc', ascending=False)

,checkpoint,valid_acc,test_acc,config
33,outputs/bert_concat_2/checkpoint-492-epoch-6,0.782344,0.762712,"{'overwrite_output_dir': True, 'num_train_epoc..."
27,outputs/bert_concat_2/checkpoint-410-epoch-5,0.777778,0.762712,"{'overwrite_output_dir': True, 'num_train_epoc..."
48,outputs/bert_concat_4/checkpoint-246-epoch-3,0.771689,0.761501,"{'overwrite_output_dir': True, 'num_train_epoc..."
58,outputs/bert_concat_4/checkpoint-574-epoch-7,0.762557,0.750605,"{'overwrite_output_dir': True, 'num_train_epoc..."
50,outputs/bert_concat_4/checkpoint-820-epoch-10,0.761035,0.746973,"{'overwrite_output_dir': True, 'num_train_epoc..."
...,...,...,...,...
107,outputs/bert_concat_8/checkpoint-164-epoch-2,0.490107,0.487893,"{'overwrite_output_dir': True, 'num_train_epoc..."
108,outputs/bert_concat_9/checkpoint-246-epoch-3,0.490107,0.487893,"{'overwrite_output_dir': True, 'num_train_epoc..."
111,outputs/bert_concat_9/checkpoint-410-epoch-5,0.490107,0.487893,"{'overwrite_output_dir': True, 'num_train_epoc..."
115,outputs/bert_concat_9/checkpoint-328-epoch-4,0.490107,0.487893,"{'overwrite_output_dir': True, 'num_train_epoc..."


In [7]:
# From above, we select outputs/bert_concat_2/checkpoint-492-epoch-6 as the model with the best score on the validation dataset.
import sklearn
from simpletransformers.classification import ClassificationModel

model = ClassificationModel("bert", "outputs/bert_concat_2/checkpoint-492-epoch-6")
model.eval_model(test_dataset, acc=sklearn.metrics.accuracy_score)

  0%|          | 0/826 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/104 [00:00<?, ?it/s]

({'mcc': 0.5263185681854179,
  'tp': 315,
  'tn': 315,
  'fp': 88,
  'fn': 108,
  'auroc': 0.827687145463398,
  'auprc': 0.8480503719450359,
  'acc': 0.7627118644067796,
  'eval_loss': 0.77070051836423},
 array([[-2.65237904,  2.87104917],
        [-2.32644939,  2.04971766],
        [ 1.69096422, -2.29835248],
        ...,
        [-2.6310544 ,  2.7471025 ],
        [-2.61189628,  2.66762495],
        [-2.60511088,  2.63315749]]),
 [])

In [6]:
for _, i in [i for i in df[df['checkpoint'].str.contains('epoch')].sort_values('valid_acc', ascending=False).iterrows()][:5]:
    print(i['valid_acc'], i['config'])

0.7823439878234398 {'overwrite_output_dir': True, 'num_train_epochs': 10, 'fp16': False, 'train_batch_size': 8, 'gradient_accumulation_steps': 4, 'evaluate_during_training': True, 'max_seq_length': 512, 'learning_rate': 4e-05, 'early_stopping_consider_epochs': True, 'early_stopping_delta': 0.01, 'early_stopping_metric': 'acc', 'early_stopping_metric_minimize': False, 'early_stopping_patience': 3, 'evaluate_during_training_steps': 331, 'output_dir': 'outputs/', 'warmup_ratio': 0.06, 'save_steps': 2000, 'manual_seed': 12345}
0.7777777777777778 {'overwrite_output_dir': True, 'num_train_epochs': 10, 'fp16': False, 'train_batch_size': 8, 'gradient_accumulation_steps': 4, 'evaluate_during_training': True, 'max_seq_length': 512, 'learning_rate': 4e-05, 'early_stopping_consider_epochs': True, 'early_stopping_delta': 0.01, 'early_stopping_metric': 'acc', 'early_stopping_metric_minimize': False, 'early_stopping_patience': 3, 'evaluate_during_training_steps': 331, 'output_dir': 'outputs/', 'warmu